In [155]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [156]:
# Load the data
data = pd.read_csv('Airbnb_Open_Data.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_11180\126891495.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Airbnb_Open_Data.csv')


In [157]:
data.isnull().sum()

id                                     0
NAME                                 250
host id                                0
host_identity_verified               289
host name                            406
neighbourhood group                   29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country code                         131
instant_bookable                     105
cancellation_policy                   76
room type                              0
Construction year                    214
price                                247
service fee                          273
minimum nights                       409
number of reviews                    183
last review                        15893
reviews per month                  15879
review rate number                   326
calculated host listings count       319
availability 365                     448
house_rules     

In [158]:
# Function to clean the name
def clean_name(name):
    if pd.isna(name):
        return "Unknown"
    # Convert to string, strip spaces, replace multiple spaces, and remove newlines
    name = str(name).strip()
    name = re.sub(' +', ' ', name)  # Replace multiple spaces with a single space
    name = re.sub(r'\s+', ' ', name)  # Replace newlines and tabs with a single space
    return name

data['NAME'] = data['NAME'].apply(clean_name)
data['host name'] = data['host name'].apply(clean_name)
data['house_rules'] = data['house_rules'].apply(clean_name)

# Drop the 'license' column due to high percentage of missing values
data.drop(columns=['license'], inplace=True)

# Convert 'price' and 'service fee' to numeric after removing currency symbols
data['NAME'] = data['NAME'].replace("'", '', regex=True)
data['host name'] = data['host name'].replace("'", '', regex=True)
data['price'] = data['price'].replace('[$,]', '', regex=True).astype(float)
data['service fee'] = data['service fee'].replace('[$,]', '', regex=True).astype(float)

# Handling data quality issues
data.loc[data['availability 365'] > 365, 'availability 365'] = 365
data.loc[data['availability 365'] < 0, 'availability 365'] = data.loc[data['availability 365'] < 0, 'availability 365'].abs()

In [159]:
data['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'brookln', 'manhatan', 'Queens', nan,
       'Staten Island', 'Bronx'], dtype=object)

In [160]:
data.loc[data['neighbourhood group'] == 'manhatan', 'neighbourhood group'] = 'Manhattan'
data.loc[data['neighbourhood group'] == 'brookln', 'neighbourhood group'] = 'Brooklyn'

In [161]:
data['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', nan, 'Staten Island', 'Bronx'],
      dtype=object)

In [162]:
data['country'].unique()

array(['United States', nan], dtype=object)

In [163]:
data['country code'].unique()

array(['US', nan], dtype=object)

In [164]:
data['country'].fillna('United States', inplace=True)

In [165]:
data['country code'].fillna('US', inplace=True)

In [166]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified              289
host name                             0
neighbourhood group                  29
neighbourhood                        16
lat                                   8
long                                  8
country                               0
country code                          0
instant_bookable                    105
cancellation_policy                  76
room type                             0
Construction year                   214
price                               247
service fee                         273
minimum nights                      409
number of reviews                   183
last review                       15893
reviews per month                 15879
review rate number                  326
calculated host listings count      319
availability 365                    448
house_rules                           0


In [167]:
data.duplicated().sum()

541

In [168]:
data.drop_duplicates(inplace=True)

In [169]:
data.duplicated().sum()

0

In [170]:
def fill_na_with_distribution(data, column):
    # Get the value counts (distribution) of non-null values
    value_counts = data[column].value_counts(normalize=True)
    
    # Get the number of NaNs in the column
    num_nans = data[column].isna().sum()
    
    # Calculate the number of values to fill based on the distribution
    fill_values = np.random.choice(
        value_counts.index, 
        size=num_nans, 
        p=value_counts.values
    )
    
    # Fill NaN values with the sampled values
    data.loc[data[column].isna(), column] = fill_values

# Apply this function to the columns you want to fill
columns_to_fill = ['lat', 'long', 'cancellation_policy']  # Replace with your column names
for column in columns_to_fill:
    fill_na_with_distribution(data, column)


In [171]:
median_price = data['price'].median()
median_price

625.0

In [172]:
median_year = data['Construction year'].median()
median_year

2012.0

In [173]:
median_fee = data['service fee'].median()
median_fee

125.0

In [174]:
median_nights = data['minimum nights'].median()
median_nights

3.0

In [175]:
median_reviews = data['number of reviews'].median()
median_reviews

7.0

In [176]:
median_availability = data['availability 365'].median()
median_availability

96.0

In [177]:
median_listing = data['calculated host listings count'].median()
median_listing

1.0

In [178]:
data['price'].fillna(median_price, inplace=True)

In [179]:
data['Construction year'].fillna(median_year, inplace=True)

In [180]:
data['service fee'].fillna(median_fee, inplace=True)

In [181]:
data['minimum nights'].fillna(median_nights, inplace=True)

In [182]:
data['number of reviews'].fillna(median_reviews, inplace=True)

In [183]:
data['availability 365'].fillna(median_availability, inplace=True)

In [184]:
data['calculated host listings count'].fillna(median_listing, inplace=True)

In [185]:
data['last review'] = pd.to_datetime(data['last review'])

In [186]:
median_last_review = data['last review'].median()

In [187]:
data['last review'].fillna(median_last_review, inplace=True)

In [188]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified              289
host name                             0
neighbourhood group                  29
neighbourhood                        16
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                    105
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                 15818
review rate number                  319
calculated host listings count        0
availability 365                      0
house_rules                           0


In [189]:
# Encode categorical variable using LabelEncoder
label_encoder = LabelEncoder()
data['cancellation_policy_encoded'] = label_encoder.fit_transform(data['cancellation_policy'])

# Define the features and target variable
features = ['price', 'lat', 'long', 'cancellation_policy_encoded']
target = 'host_identity_verified'

# Split the data into training and prediction sets
train_data = data.dropna(subset=[target])  # Rows without NaN in target
test_data = data[data[target].isna()]        # Rows with NaN in target

# Separate features and target variable for training data
X_train = train_data[features]
y_train = train_data[target]

# Separate features for test data
X_test = test_data[features]

# Encode target variable 'host_identity_verified' using LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train_encoded)

# Predict the NaN values
predicted_values_encoded = model.predict(X_test)

# Decode the predicted values
predicted_values = target_encoder.inverse_transform(predicted_values_encoded)

# Assign predicted values back to the original DataFrame
data.loc[data[target].isna(), target] = predicted_values

# Drop the encoded cancellation_policy column if it was added
data.drop(columns=['cancellation_policy_encoded'], inplace=True)

# Check the result
print(data[target].value_counts())

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
print(f'\nTraining Accuracy: {train_accuracy:.2f}')


host_identity_verified
unconfirmed    51118
verified       50940
Name: count, dtype: int64

Training Accuracy: 0.50


In [190]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                  29
neighbourhood                        16
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                    105
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                 15818
review rate number                  319
calculated host listings count        0
availability 365                      0
house_rules                           0


## Vlookup

In [191]:
# Step 1: Create a mapping from neighbourhood to neighbourhood group using known values
neighbourhood_to_group = data.dropna(subset=['neighbourhood group']).drop_duplicates(subset=['neighbourhood']).set_index('neighbourhood')['neighbourhood group'].to_dict()

# Step 2: Define a function to fill the missing values
def fill_neighbourhood_group(row):
    if pd.isna(row['neighbourhood group']):
        return neighbourhood_to_group.get(row['neighbourhood'], row['neighbourhood group'])
    return row['neighbourhood group']

# Step 3: Apply the function to fill the missing values
data['neighbourhood group'] = data.apply(fill_neighbourhood_group, axis=1)

In [192]:
# Encode categorical variable using LabelEncoder
label_encoder = LabelEncoder()
data['neighbourhood_group_encoded'] = label_encoder.fit_transform(data['neighbourhood group'])

# Define the features and target variable
features = ['lat', 'neighbourhood_group_encoded','long']
target = 'neighbourhood'

# Split the data into training and prediction sets
train_data = data.dropna(subset=[target])  # Rows without NaN in target
test_data = data[data[target].isna()]        # Rows with NaN in target

# Separate features and target variable for training data
X_train = train_data[features]
y_train = train_data[target]

# Separate features for test data
X_test = test_data[features]

# Encode target variable 'host_identity_verified' using LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train_encoded)

# Predict the NaN values
predicted_values_encoded = model.predict(X_test)

# Decode the predicted values
predicted_values = target_encoder.inverse_transform(predicted_values_encoded)

# Assign predicted values back to the original DataFrame
data.loc[data[target].isna(), target] = predicted_values

# Check the result
print(data[target].value_counts())

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
print(f'\nTraining Accuracy: {train_accuracy:.2f}')

neighbourhood
Bedford-Stuyvesant        7901
Williamsburg              7737
Harlem                    5445
Bushwick                  4956
Hell's Kitchen            3951
                          ... 
Willowbrook                  3
Gerritsen Beach              3
Fort Wadsworth               2
Glen Oaks                    2
Chelsea, Staten Island       1
Name: count, Length: 224, dtype: int64

Training Accuracy: 0.20


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [193]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                    105
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                 15818
review rate number                  319
calculated host listings count        0
availability 365                      0
house_rules                           0


In [194]:
# Encode categorical variable using LabelEncoder
label_encoder = LabelEncoder()
data['cancellation_policy_encoded'] = label_encoder.fit_transform(data['cancellation_policy'])
data['host_identity_verified_encoded'] = label_encoder.fit_transform(data['host_identity_verified'])

# Define the features and target variable
features = ['host_identity_verified_encoded', 'cancellation_policy_encoded', 'price']
target = 'instant_bookable'

# Split the data into training and prediction sets
train_data = data.dropna(subset=[target])  # Rows without NaN in target
test_data = data[data[target].isna()]        # Rows with NaN in target

# Separate features and target variable for training data
X_train = train_data[features]
y_train = train_data[target]

# Separate features for test data
X_test = test_data[features]

# Encode target variable 'host_identity_verified' using LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train_encoded)

# Predict the NaN values
predicted_values_encoded = model.predict(X_test)

# Decode the predicted values
predicted_values = target_encoder.inverse_transform(predicted_values_encoded)

# Assign predicted values back to the original DataFrame
data.loc[data[target].isna(), target] = predicted_values

# Drop the encoded cancellation_policy column if it was added
data.drop(columns=['cancellation_policy_encoded', 'host_identity_verified_encoded'], inplace=True)

# Check the result
print(data[target].value_counts())

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
print(f'\nTraining Accuracy: {train_accuracy:.2f}')

instant_bookable
False    51260
True     50798
Name: count, dtype: int64

Training Accuracy: 0.50


In [195]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                      0
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                 15818
review rate number                  319
calculated host listings count        0
availability 365                      0
house_rules                           0


In [196]:
# Encode categorical variable using LabelEncoder
label_encoder = LabelEncoder()
data['cancellation_policy_encoded'] = label_encoder.fit_transform(data['cancellation_policy'])

# Define the features and target variable
features = ['price', 'Construction year', 'number of reviews','cancellation_policy_encoded', 'lat', 'long']
target = 'review rate number'

# Split the data into training and prediction sets
train_data = data.dropna(subset=[target])  # Rows without NaN in target
test_data = data[data[target].isna()]        # Rows with NaN in target

# Separate features and target variable for training data
X_train = train_data[features]
y_train = train_data[target]

# Separate features for test data
X_test = test_data[features]

# Encode target variable 'host_identity_verified' using LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train_encoded)

# Predict the NaN values
predicted_values_encoded = model.predict(X_test)

# Decode the predicted values
predicted_values = target_encoder.inverse_transform(predicted_values_encoded)

# Assign predicted values back to the original DataFrame
data.loc[data[target].isna(), target] = predicted_values

# Drop the encoded cancellation_policy column if it was added
data.drop(columns=['cancellation_policy_encoded'], inplace=True)

# Check the result
print(data[target].value_counts())

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
print(f'\nTraining Accuracy: {train_accuracy:.2f}')

review rate number
5.0    23360
4.0    23283
3.0    23163
2.0    23066
1.0     9186
Name: count, dtype: int64

Training Accuracy: 0.23


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [197]:
data.isnull().sum()

id                                    0
NAME                                  0
host id                               0
host_identity_verified                0
host name                             0
neighbourhood group                   0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
country code                          0
instant_bookable                      0
cancellation_policy                   0
room type                             0
Construction year                     0
price                                 0
service fee                           0
minimum nights                        0
number of reviews                     0
last review                           0
reviews per month                 15818
review rate number                    0
calculated host listings count        0
availability 365                      0
house_rules                           0


In [198]:
# Encode categorical variable using LabelEncoder
label_encoder = LabelEncoder()
data['host_identity_verified_encoded'] = label_encoder.fit_transform(data['host_identity_verified'])

# Define the features and target variable
features = ['host_identity_verified_encoded', 'price', 'number of reviews', 'lat', 'long']
target = 'reviews per month'

# Split the data into training and prediction sets
train_data = data.dropna(subset=[target])  # Rows without NaN in target
test_data = data[data[target].isna()]        # Rows with NaN in target

# Separate features and target variable for training data
X_train = train_data[features]
y_train = train_data[target]

# Separate features for test data
X_test = test_data[features]

# Encode target variable 'host_identity_verified' using LabelEncoder
target_encoder = LabelEncoder()
y_train_encoded = target_encoder.fit_transform(y_train)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train_encoded)

# Predict the NaN values
predicted_values_encoded = model.predict(X_test)

# Decode the predicted values
predicted_values = target_encoder.inverse_transform(predicted_values_encoded)

# Assign predicted values back to the original DataFrame
data.loc[data[target].isna(), target] = predicted_values

# Drop the encoded cancellation_policy column if it was added
data.drop(columns=['host_identity_verified_encoded'], inplace=True)

# Check the result
print(data[target].value_counts())

# Predict on the training data
y_train_pred = model.predict(X_train)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
print(f'\nTraining Accuracy: {train_accuracy:.2f}')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


reviews per month
0.03     17338
1.00      1579
0.05      1491
0.09      1264
0.04      1260
         ...  
9.10         1
10.36        1
8.50         1
15.32        1
33.08        1
Name: count, Length: 1016, dtype: int64

Training Accuracy: 0.03


In [199]:
data.isnull().sum()

id                                0
NAME                              0
host id                           0
host_identity_verified            0
host name                         0
neighbourhood group               0
neighbourhood                     0
lat                               0
long                              0
country                           0
country code                      0
instant_bookable                  0
cancellation_policy               0
room type                         0
Construction year                 0
price                             0
service fee                       0
minimum nights                    0
number of reviews                 0
last review                       0
reviews per month                 0
review rate number                0
calculated host listings count    0
availability 365                  0
house_rules                       0
neighbourhood_group_encoded       0
dtype: int64

In [200]:
data.columns

Index(['id', 'NAME', 'host id', 'host_identity_verified', 'host name',
       'neighbourhood group', 'neighbourhood', 'lat', 'long', 'country',
       'country code', 'instant_bookable', 'cancellation_policy', 'room type',
       'Construction year', 'price', 'service fee', 'minimum nights',
       'number of reviews', 'last review', 'reviews per month',
       'review rate number', 'calculated host listings count',
       'availability 365', 'house_rules', 'neighbourhood_group_encoded'],
      dtype='object')

In [201]:
# Rename columns to avoid SQL syntax issues
data.columns = data.columns.str.strip().str.replace(' ', '_').str.lower()

In [202]:
data.columns

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'neighbourhood_group_encoded'],
      dtype='object')

In [203]:
data.to_csv('final_dataset.csv')